**Import Libraries**

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os
from stem import Signal
from stem.control import Controller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

**Add Socks proxy for proxy rotation**

In [2]:
proxies = {
    'http': 'socks5://127.0.0.1:9050',
    'https': 'socks5://127.0.0.1:9050'
}

**Generate hashed password and edit /etc/tor/torrc and /usr/share/tor/tor-service-defaults-torrc file and add 
<br>ControlPort 9051
<br>CookieAuthentication 1
<br>CookieAuthFileGroupReadable 1
<br>HashedControlPassword 'Put output of tor --hash-password 'put your password here''
<br>and go to /var/run/tor and delete all files
<br>then run sudo /etc/init.d/tor restart 
<br>it will generate new files which will be used below**

In [3]:
with Controller.from_port(port = 9051) as c:
    c.authenticate('Put password here') #tor --hash-password 'put password here'
    c.signal(Signal.NEWNYM)


url_raw = requests.get('https://www.bigbasket.com/product/all-categories/',proxies=proxies).text

In [5]:
html_content = bs(url_raw,'html.parser') #Generate raw html page source

In [6]:
test = html_content.find_all('a',class_='bdr-btm',href=True) #Get the categories of items

In [7]:
cat_list = []
for i in test:
    cat_list.append(i['href'])

In [210]:
cat_list #List of categories

['/cl/fruits-vegetables/',
 '/cl/foodgrains-oil-masala/',
 '/cl/bakery-cakes-dairy/',
 '/cl/beverages/',
 '/cl/snacks-branded-foods/',
 '/cl/beauty-hygiene/',
 '/cl/cleaning-household/',
 '/cl/kitchen-garden-pets/',
 '/cl/eggs-meat-fish/',
 '/cl/gourmet-world-food/',
 '/cl/baby-care/',
 '/cl/fruits-vegetables/',
 '/cl/foodgrains-oil-masala/',
 '/cl/bakery-cakes-dairy/',
 '/cl/beverages/',
 '/cl/snacks-branded-foods/',
 '/cl/beauty-hygiene/',
 '/cl/cleaning-household/',
 '/cl/kitchen-garden-pets/',
 '/cl/eggs-meat-fish/',
 '/cl/gourmet-world-food/',
 '/cl/baby-care/']

**Run tor with rotating proxy for each category and scroll down to the end of page untill end of page is reached
<br> then after page 6 we need to click "Show More" button and run end of page code
<br> For different IP different list of items are returned and all items in the category is not returned
<br> Finally, we put it in pandas dataframe and export as csv file**

In [278]:
for cat in tqdm(cat_list[6:11]):
    with Controller.from_port(port = 9051) as c:
        c.authenticate('Put password here')
        c.signal(Signal.NEWNYM)
    profile = FirefoxProfile(r'/home/neerad/Downloads/tor-browser_en-US/Browser/TorBrowser/Data/Browser/profile.default')
    profile.set_preference('network.proxy.type', 1)
    profile.set_preference('network.proxy.socks', '127.0.0.1')
    profile.set_preference('network.proxy.socks_port', 9050)
    profile.set_preference("network.proxy.socks_remote_dns", False)
    profile.update_preferences()
    driver = webdriver.Firefox(firefox_profile= profile, executable_path=r'/home/neerad/Downloads/geckodriver')
    driver.get("https://www.bigbasket.com"+cat+"?nc=cs")
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        try:
            driver.find_elements_by_xpath('//div[@class="show-more"]')[0].find_element_by_css_selector('button').click()
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        except:
            pass
        if lastCount==lenOfPage:
            match=True
            main = driver.find_elements_by_xpath('//*[@id="dynamicDirective"]/product-deck/section/div[2]/div[4]/div[1]/div/div/div[2]')
            brand = [i.text for i in main[0].find_elements_by_xpath('//div[@class="col-sm-12 col-xs-7 prod-name"]//h6[@class="ng-binding"]')]
            name = [j.text for j in main[0].find_elements_by_xpath('//div[@class="col-sm-12 col-xs-7 prod-name"]//a')]
            quant = [k.text for k in main[0].find_elements_by_xpath('//div[@class="col-sm-12 col-xs-7 qnty-selection"]//span[@data-bind="label"]//span[@class="ng-binding"][1]')]
            price = [l.text for l in main[0].find_elements_by_xpath('//div[@class="po-markup"]//span[@class="discnt-price"]//span[@class="ng-binding"]')]
            globals()['df_'+cat[4:-1].replace('-','_')] = pd.DataFrame({'Brand':brand,'Item_name':name,'Quantity':quant,'Price in Rupees':price})
    time.sleep(5)
    driver.quit()

        

In [272]:
df_bakery_cakes_dairy.to_csv('bakery_cakes_dairy.csv')

In [273]:
df_bakery_cakes_dairy

,Brand,Item_name,Quantity,Price in Rupees
0,Nandini GoodLife,Toned Milk,500 ml,25
1,Milky Mist,Paneer - Premium Fresh,500 g,188
2,Amul,Butter - Pasteurised,100 g,48
3,Milky Mist,Curd - Farm Fresh,2x500 g,70
4,Fresho,"Bread - Whole Wheat, Chemical Free 400G + Eggs...",Combo,188
...,...,...,...,...
718,Baskin Robbins,Ice Cream Bar - Cookies n Cream,65 ml,65
719,MOOZ,Sour Cream Peri Peri,150 g,220
720,Baskin Robbins,Ice Cream Bar - Almonds n Caramel,65 ml,65
721,Baskin Robbins,Ice Cream - Very Berry Strawberry,450 ml,275


In [285]:
df_baby_care.to_csv('baby_care.csv')